In [5]:
import requests
import pandas as pd

API_KEY = "9f3f8e86cb59b046c3ce7459fff21695"

url = f"https://gnews.io/api/v4/top-headlines?lang=en&country=in&max=50&apikey={API_KEY}"

response = requests.get(url)
data = response.json()

articles = data["articles"]

df = pd.DataFrame(articles)
df = df[["title", "description", "content"]]

df.head()


,title,description,content
0,Tom Banton fifty steers DLS chase to seal seri...,"SL v ENG 2025/26, Sri Lanka vs England Match R...",Harry Brook's blazing 12-ball cameo tips game ...
1,Budget numbers show Government is abandoning i...,The union government wants to play a less impo...,Finance Minister Nirmala Sitharaman's ninth bu...
2,Man City player ratings vs Tottenham: Advantag...,GOAL rates Manchester City's players from thei...,It took only 11 minutes for City to open the s...
3,Michael Carrick hails Man Utd's unsung hero af...,Benjamin Sesko came off the bench and scored M...,Benjamin Sesko came off the bench and scored M...
4,CM Omar to chair cabinet meet on Monday,Key policy and administrative issues are expec...,"Srinagar, Feb 01: Chief Minister Omar Abdullah..."


In [7]:
df["text"] = df["title"] + " " + df["description"]
df["text"].head()



0    Tom Banton fifty steers DLS chase to seal seri...
1    Budget numbers show Government is abandoning i...
2    Man City player ratings vs Tottenham: Advantag...
3    Michael Carrick hails Man Utd's unsung hero af...
4    CM Omar to chair cabinet meet on Monday Key po...
Name: text, dtype: object

In [8]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)     # remove links
    text = re.sub(r"[^a-zA-Z ]", "", text)  # remove special characters
    return text

df["clean_text"] = df["text"].apply(clean_text)
df["clean_text"].head()


0    tom banton fifty steers dls chase to seal seri...
1    budget numbers show government is abandoning i...
2    man city player ratings vs tottenham advantage...
3    michael carrick hails man utds unsung hero aft...
4    cm omar to chair cabinet meet on monday key po...
Name: clean_text, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [10]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=1000
)

X = vectorizer.fit_transform(df["clean_text"])

X.shape


(10, 202)

In [11]:
from sklearn.cluster import KMeans


In [12]:
k = 3  # number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)

df["cluster"] = kmeans.fit_predict(X)

df[["title", "cluster"]]
for i in range(k):
    print(f"\nCluster {i}")
    print(df[df["cluster"] == i]["title"].head(3).values)



Cluster 0
['Man City player ratings vs Tottenham: Advantage Arsenal - again! Erling Haaland endures another no-show as Cityzens throw away win against Dominic Solanke-inspired Spurs'
 "Michael Carrick hails Man Utd's unsung hero after Benjamin Sesko takes headlines"
 'Carrick learns harsh Casemiro lesson in bigger test than Arsenal and Man City']

Cluster 1
["Budget numbers show Government is abandoning its kartavya and that's not good news"
 'CM Omar to chair cabinet meet on Monday'
 'Elections In 3 Months, But Ignores In Budget?']

Cluster 2
['Tom Banton fifty steers DLS chase to seal series for England'
 'Xbox reveals next week’s game releases for 2 to 6 February: Dragon Quest, Carmageddon and more']
